In [118]:
import pandas as pd
import json
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import URL

In [119]:
pd.options.display.max_columns = 100

In [120]:
username = 'root'
password='root'
host='localhost'
database='jmdb'

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")


In [121]:
df = pd.read_csv("emdb.csv",encoding="utf-16", sep = ";")

In [122]:
df = df.rename(columns = {v : v.strip() for v in df.columns})

In [123]:
print(df.columns)

Index(['Numéro', 'Titre', 'Titre alternatif', 'Année', 'Pays', 'Genre(s)',
       'Langues', 'Durée', 'Chemin de l'affiche', 'Réalisateur(s)',
       'Scénariste(s)', 'Compositeur(s)', 'Studio', 'Distribution',
       'Bande-annonce', 'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
       'Commentaires', 'Etiquettes', 'Support', 'Taille du fichier',
       'ID TheMovieDb', 'ID IMDb'],
      dtype='object')


# table movie , director et intermediaire

In [124]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [125]:
# Modifier le chemin de l'affiche pour ne conserver que la dernière partie
df_movie["Chemin de l'affiche"] = df_movie["Chemin de l'affiche"].astype(str).apply(lambda x: x.split('\\')[-1])

# Afficher le DataFrame résultant
print(df_movie.head())  # Affiche les premières lignes du DataFrame

   Numéro                            Titre  \
0    3635                         Vermines   
1    3634  Les derniers samouraïs (vostfr)   
2    3633            Un homme nommé Sledge   
3    3632           Sanglantes confessions   
4    3631                           Racket   

                        Titre alternatif Année  Durée Chemin de l'affiche  \
0                                    NaN  2023    103          003719.jpg   
1               狼よ落日を斬れ (Titre original)  1974    159          003718.jpg   
2   A Man Called Sledge (Titre original)  1970     93          003717.jpg   
3      True Confessions (Titre original)  1981    108          003716.jpg   
4  The Long Good Friday (Titre original)  1980    114          003715.jpg   

                         Bande-annonce  \
0  www.youtube.com/watch?v=vvmsk4CjwIk   
1                                  NaN   
2  www.youtube.com/watch?v=Uv430fWuTjU   
3  www.youtube.com/watch?v=iqk_iCcq6PM   
4  www.youtube.com/watch?v=B90zNzyFk-w   

       

In [126]:
list_director_movie = ["Réalisateur(s)", 'Numéro']
df_director_movie = df.copy()[list_director_movie]

In [127]:
# df_director_movie ["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",")
df_director_movie["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",\s*")

In [128]:
df_director_movie  = df_director_movie.explode("Réalisateur(s)")

In [129]:
df_director  = df_director_movie.explode("Réalisateur(s)").drop_duplicates("Réalisateur(s)").reset_index(drop=True).reset_index()
df_director = df_director.rename(columns={"index":"id"}).drop(columns="Numéro")

#.reset_index().rename(columns={"index":"id"})

In [130]:
df_intermediaire_movie_director = df_director.merge(df_director_movie, on = "Réalisateur(s)").drop(columns="Réalisateur(s)")

In [131]:
# Remplacer les valeurs NULL de la colonne 'id' par 0
df_director['id'] = df_director['id'].fillna(0)
# Convertir la colonne 'id' en type entier
df_director['id'] = df_director['id'].astype(int)

# Réorganiser les ID si nécessaire pour éviter les conflits
max_id = df_director['id'].max()
if max_id > 0:
    df_director['id'] = df_director['id'] + 1

In [132]:
display(df_director)

,id,Réalisateur(s)
0,1,Sébastien Vaniček
1,2,Kenji Misumi
2,3,Giorgio Gentili
3,4,Vic Morrow
4,5,Ulu Grosbard
...,...,...
1778,1779,Richard C. Sarafian
1779,1780,Chris Petit
1780,1781,Brett Ratner
1781,1782,Buster Keaton


In [133]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3635,Vermines,NaN,2023,103,003719.jpg,www.youtube.com/watch?v=vvmsk4CjwIk,NaN,Kaleb est un jeune adulte qui vit de petits bo...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Vermines ...,Matroska (MKV),NaN,Fichier multimédia,2.39 GB,movie/1063879,26744289.0
1,3634,Les derniers samouraïs (vostfr),狼よ落日を斬れ (Titre original),1974,159,003718.jpg,NaN,NaN,"A la fin du 19ème siècle, un samourai (Sugi To...",[My Book]F:\#DIVX\DIVX MOVIEZ\CHANBARA\The Las...,Matroska (MKV),NaN,Fichier multimédia,1.80 GB,movie/70385,123129.0
2,3633,Un homme nommé Sledge,A Man Called Sledge (Titre original),1970,93,003717.jpg,www.youtube.com/watch?v=Uv430fWuTjU,"Quand SLEDGE frappe la ville, il reste frappé !","Sledge, un célèbre bandit de l'Ouest, apprend ...",[My Book]F:\#DIVX\DIVX MOVIEZ\WESTERN\Un Homme...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/70387,66050.0
3,3632,Sanglantes confessions,True Confessions (Titre original),1981,108,003716.jpg,www.youtube.com/watch?v=iqk_iCcq6PM,Two brothers out to save everyone else. But t...,Tom et Desmond Spellacy sont frères mais ont s...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Sanglantes...,Matroska (MKV),NaN,Fichier multimédia,2.33 GB,movie/32077,83232.0
4,3631,Racket,The Long Good Friday (Titre original),1980,114,003715.jpg,www.youtube.com/watch?v=B90zNzyFk-w,Who lit the fuse that tore Harold's world apart?,"Harold, un gangster anglais prospère, s'apprêt...",[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Racket Du ...,Matroska (MKV),NaN,Fichier multimédia,2.35 GB,movie/14807,81070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3630,5,le mécano de la général,The General (Titre original),1926,79,000005.jpg,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3631,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,000004.jpg,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3632,3,Les feux de la rampe,Limelight (Titre original),1952,137,000003.jpg,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3633,2,Les temps modernes,Modern Times (Titre original),1936,87,000002.jpg,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [134]:
display(df_intermediaire_movie_director)

,id,Numéro
0,0,3635
1,1,3634
2,1,3301
3,1,3299
4,1,3298
...,...,...
3913,1778,20
3914,1779,18
3915,1780,15
3916,1781,5


In [135]:
# Trouver l'index de la ligne avec le 'Réalisateur(s)' NULL
null_director_index = df_director[df_director['Réalisateur(s)'].isnull()].index

# Si une ligne avec 'Réalisateur(s)' NULL est trouvée
if not null_director_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_director.loc[null_director_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_director['id'].max()
    if max_id > 0:
        df_director.loc[df_director.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_director.head())

# Enregistrer le DataFrame modifié dans la table 'director' de la base de données
df_director.to_sql("director", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_director = df_director.merge(df_director_movie, on="Réalisateur(s)").drop(columns="Réalisateur(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_director.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_director.to_sql("movie_director", index=False, con=engine, if_exists="replace")

     id     Réalisateur(s)
0  1784  Sébastien Vaniček
1     2       Kenji Misumi
2     3    Giorgio Gentili
3     4         Vic Morrow
4     5       Ulu Grosbard
     id  Numéro
0  1784    3635
1     2    3634
2     2    3301
3     2    3299
4     2    3298


3918

In [136]:
# df_director.to_sql("director",index=False,con = engine,if_exists = "replace" )

In [137]:
# df_intermediaire_movie_director.to_sql("movie_director",index=False,con = engine,if_exists = "replace" )

In [138]:
df_movie.to_sql("movies",index=False,con = engine,if_exists = "replace" )

3635

# table movie, screenwriter et intermediaire

In [139]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [140]:
list_screenwriter_movie = ["Scénariste(s)", 'Numéro']
df_screenwriter_movie = df.copy()[list_screenwriter_movie]

In [141]:
df_screenwriter_movie ["Scénariste(s)"] = df_screenwriter_movie["Scénariste(s)"].str.split(",\s*")

In [142]:
df_screenwriter_movie  = df_screenwriter_movie.explode("Scénariste(s)")

In [143]:
df_screenwriter  = df_screenwriter_movie.explode("Scénariste(s)").drop_duplicates("Scénariste(s)").reset_index(drop=True).reset_index()
df_screenwriter = df_screenwriter.rename(columns={"index":"id"}).drop(columns="Numéro")

In [144]:
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on = "Scénariste(s)").drop(columns="Scénariste(s)")

In [145]:
display(df_screenwriter)

,id,Scénariste(s)
0,0,Sébastien Vaniček
1,1,Florent Bernard
2,2,Takeo Kunihiro
3,3,Kenji Misumi
4,4,Vic Morrow
...,...,...
3895,3895,Ted Tally
3896,3896,David Veloz
3897,3897,Richard Rutowski
3898,3898,Buster Keaton


In [146]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3635,Vermines,NaN,2023,103,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=vvmsk4CjwIk,NaN,Kaleb est un jeune adulte qui vit de petits bo...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Vermines ...,Matroska (MKV),NaN,Fichier multimédia,2.39 GB,movie/1063879,26744289.0
1,3634,Les derniers samouraïs (vostfr),狼よ落日を斬れ (Titre original),1974,159,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"A la fin du 19ème siècle, un samourai (Sugi To...",[My Book]F:\#DIVX\DIVX MOVIEZ\CHANBARA\The Las...,Matroska (MKV),NaN,Fichier multimédia,1.80 GB,movie/70385,123129.0
2,3633,Un homme nommé Sledge,A Man Called Sledge (Titre original),1970,93,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=Uv430fWuTjU,"Quand SLEDGE frappe la ville, il reste frappé !","Sledge, un célèbre bandit de l'Ouest, apprend ...",[My Book]F:\#DIVX\DIVX MOVIEZ\WESTERN\Un Homme...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/70387,66050.0
3,3632,Sanglantes confessions,True Confessions (Titre original),1981,108,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=iqk_iCcq6PM,Two brothers out to save everyone else. But t...,Tom et Desmond Spellacy sont frères mais ont s...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Sanglantes...,Matroska (MKV),NaN,Fichier multimédia,2.33 GB,movie/32077,83232.0
4,3631,Racket,The Long Good Friday (Titre original),1980,114,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=B90zNzyFk-w,Who lit the fuse that tore Harold's world apart?,"Harold, un gangster anglais prospère, s'apprêt...",[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Racket Du ...,Matroska (MKV),NaN,Fichier multimédia,2.35 GB,movie/14807,81070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3630,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3631,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3632,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3633,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [147]:
display(df_intermediaire_movie_screenwriter)

,id,Numéro
0,0,3635
1,1,3635
2,2,3634
3,3,3634
4,4,3633
...,...,...
7037,3895,15
7038,3896,14
7039,3897,14
7040,3898,5


In [148]:
# Trouver l'index de la ligne avec le 'Scénariste(s)' NULL
null_screenwriter_index = df_screenwriter[df_screenwriter['Scénariste(s)'].isnull()].index

# Si une ligne avec 'Scénariste(s)' NULL est trouvée
if not null_screenwriter_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_screenwriter.loc[null_screenwriter_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_screenwriter['id'].max()
    if max_id > 0:
        df_screenwriter.loc[df_screenwriter.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_screenwriter.head())

# Enregistrer le DataFrame modifié dans la table 'screenwriter' de la base de données
df_screenwriter.to_sql("screenwriter", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on="Scénariste(s)").drop(columns="Scénariste(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_screenwriter.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter", index=False, con=engine, if_exists="replace")


     id      Scénariste(s)
0  3900  Sébastien Vaniček
1     1    Florent Bernard
2     2     Takeo Kunihiro
3     3       Kenji Misumi
4     4         Vic Morrow
     id  Numéro
0  3900    3635
1     1    3635
2     2    3634
3     3    3634
4     4    3633


7042

In [149]:
# df_screenwriter.to_sql("screenwriter",index=False,con = engine,if_exists = "replace" )

In [150]:
# df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter",index=False,con = engine,if_exists = "replace" )

# table movie, music et intermediaire

In [151]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [152]:
list_music_movie = ["Compositeur(s)", 'Numéro']
df_music_movie = df.copy()[list_music_movie]

In [153]:
df_music_movie ["Compositeur(s)"] = df_music_movie["Compositeur(s)"].str.split(",\s*")

In [154]:
df_music_movie  = df_music_movie.explode("Compositeur(s)")

In [155]:
df_music  = df_music_movie.explode("Compositeur(s)").drop_duplicates("Compositeur(s)").reset_index(drop=True).reset_index()
df_music = df_music.rename(columns={"index":"id"}).drop(columns="Numéro")

In [156]:
df_intermediaire_movie_music = df_music.merge(df_music_movie, on = "Compositeur(s)").drop(columns="Compositeur(s)")

In [157]:
display(df_music)

,id,Compositeur(s)
0,0,NaN
1,1,Akira Ifukube
2,2,Gianni Ferrio
3,3,Georges Delerue
4,4,Francis Monkman
...,...,...
1277,1277,Machine Head
1278,1278,Photek
1279,1279,John Frizzell
1280,1280,Leonard Cohen


In [158]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3635,Vermines,NaN,2023,103,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=vvmsk4CjwIk,NaN,Kaleb est un jeune adulte qui vit de petits bo...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Vermines ...,Matroska (MKV),NaN,Fichier multimédia,2.39 GB,movie/1063879,26744289.0
1,3634,Les derniers samouraïs (vostfr),狼よ落日を斬れ (Titre original),1974,159,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"A la fin du 19ème siècle, un samourai (Sugi To...",[My Book]F:\#DIVX\DIVX MOVIEZ\CHANBARA\The Las...,Matroska (MKV),NaN,Fichier multimédia,1.80 GB,movie/70385,123129.0
2,3633,Un homme nommé Sledge,A Man Called Sledge (Titre original),1970,93,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=Uv430fWuTjU,"Quand SLEDGE frappe la ville, il reste frappé !","Sledge, un célèbre bandit de l'Ouest, apprend ...",[My Book]F:\#DIVX\DIVX MOVIEZ\WESTERN\Un Homme...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/70387,66050.0
3,3632,Sanglantes confessions,True Confessions (Titre original),1981,108,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=iqk_iCcq6PM,Two brothers out to save everyone else. But t...,Tom et Desmond Spellacy sont frères mais ont s...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Sanglantes...,Matroska (MKV),NaN,Fichier multimédia,2.33 GB,movie/32077,83232.0
4,3631,Racket,The Long Good Friday (Titre original),1980,114,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=B90zNzyFk-w,Who lit the fuse that tore Harold's world apart?,"Harold, un gangster anglais prospère, s'apprêt...",[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Racket Du ...,Matroska (MKV),NaN,Fichier multimédia,2.35 GB,movie/14807,81070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3630,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3631,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3632,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3633,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [159]:
display(df_intermediaire_movie_music)

,id,Numéro
0,0,3635
1,0,3630
2,0,3628
3,0,3627
4,0,3626
...,...,...
4022,1277,46
4023,1278,46
4024,1279,42
4025,1280,14


In [160]:
# Trouver l'index de la ligne avec le 'Compositeur(s)' NULL
null_music_index = df_music[df_music['Compositeur(s)'].isnull()].index

# Si une ligne avec 'Compositeur(s)' NULL est trouvée
if not null_music_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_music.loc[null_music_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_music['id'].max()
    if max_id > 0:
        df_music.loc[df_music.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_music.head())

# Enregistrer le DataFrame modifié dans la table 'music' de la base de données
df_music.to_sql("music", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_music = df_music.merge(df_music_movie, on="Compositeur(s)").drop(columns="Compositeur(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_music.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_music.to_sql("movie_music", index=False, con=engine, if_exists="replace")


     id   Compositeur(s)
0  1282              NaN
1     1    Akira Ifukube
2     2    Gianni Ferrio
3     3  Georges Delerue
4     4  Francis Monkman
     id  Numéro
0  1282    3635
1  1282    3630
2  1282    3628
3  1282    3627
4  1282    3626


4027

In [161]:
# df_music.to_sql("music",index=False,con = engine,if_exists = "replace" )

In [162]:
df_intermediaire_movie_music.to_sql("movie_music",index=False,con = engine,if_exists = "replace" )

4027

# table movie, studio et intermediaire

In [163]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [164]:
list_studio_movie = ["Studio", 'Numéro']
df_studio_movie = df.copy()[list_studio_movie]

In [165]:
df_studio_movie ["Studio"] = df_studio_movie["Studio"].str.split(",\s*")

In [166]:
df_studio_movie  = df_studio_movie.explode("Studio")

In [167]:
df_studio  = df_studio_movie.explode("Studio").drop_duplicates("Studio").reset_index(drop=True).reset_index()
df_studio = df_studio.rename(columns={"index":"id"}).drop(columns="Numéro")

In [168]:
df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on = "Studio").drop(columns="Studio")

In [169]:
display(df_studio)

,id,Studio
0,0,My Box Films
1,1,Shochiku
2,2,Dino de Laurentiis Cinematografica
3,3,Winkler Films
4,4,Handmade Films
...,...,...
959,959,A-Film Distribution
960,960,TFC
961,961,Dybex
962,962,British Empire Films Australia


In [170]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3635,Vermines,NaN,2023,103,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=vvmsk4CjwIk,NaN,Kaleb est un jeune adulte qui vit de petits bo...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Vermines ...,Matroska (MKV),NaN,Fichier multimédia,2.39 GB,movie/1063879,26744289.0
1,3634,Les derniers samouraïs (vostfr),狼よ落日を斬れ (Titre original),1974,159,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"A la fin du 19ème siècle, un samourai (Sugi To...",[My Book]F:\#DIVX\DIVX MOVIEZ\CHANBARA\The Las...,Matroska (MKV),NaN,Fichier multimédia,1.80 GB,movie/70385,123129.0
2,3633,Un homme nommé Sledge,A Man Called Sledge (Titre original),1970,93,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=Uv430fWuTjU,"Quand SLEDGE frappe la ville, il reste frappé !","Sledge, un célèbre bandit de l'Ouest, apprend ...",[My Book]F:\#DIVX\DIVX MOVIEZ\WESTERN\Un Homme...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/70387,66050.0
3,3632,Sanglantes confessions,True Confessions (Titre original),1981,108,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=iqk_iCcq6PM,Two brothers out to save everyone else. But t...,Tom et Desmond Spellacy sont frères mais ont s...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Sanglantes...,Matroska (MKV),NaN,Fichier multimédia,2.33 GB,movie/32077,83232.0
4,3631,Racket,The Long Good Friday (Titre original),1980,114,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=B90zNzyFk-w,Who lit the fuse that tore Harold's world apart?,"Harold, un gangster anglais prospère, s'apprêt...",[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Racket Du ...,Matroska (MKV),NaN,Fichier multimédia,2.35 GB,movie/14807,81070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3630,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3631,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3632,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3633,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [171]:
display(df_intermediaire_movie_studio)

,id,Numéro
0,0,3635
1,1,3634
2,1,3415
3,1,3409
4,1,3322
...,...,...
3632,959,172
3633,960,171
3634,961,169
3635,962,108


In [172]:
# Trouver l'index de la ligne avec le 'Studio' NULL
null_studio_index = df_studio[df_studio['Studio'].isnull()].index

# Si une ligne avec 'Studio' NULL est trouvée
if not null_studio_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_studio.loc[null_studio_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_studio['id'].max()
    if max_id > 0:
        df_studio.loc[df_studio.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_studio.head())

# Enregistrer le DataFrame modifié dans la table 'studio' de la base de données
df_studio.to_sql("studio", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on="Studio").drop(columns="Studio")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_studio.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_studio.to_sql("movie_studio", index=False, con=engine, if_exists="replace")


    id                              Studio
0  964                        My Box Films
1    1                            Shochiku
2    2  Dino de Laurentiis Cinematografica
3    3                       Winkler Films
4    4                      Handmade Films
    id  Numéro
0  964    3635
1    1    3634
2    1    3415
3    1    3409
4    1    3322


3637

In [173]:
# df_studio.to_sql("studio",index=False,con = engine,if_exists = "replace" )

In [174]:
df_intermediaire_movie_studio.to_sql("movie_studio",index=False,con = engine,if_exists = "replace" )

3637

# table movie, casting & intermediaire

In [175]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [176]:
list_casting_movie = ["Distribution", 'Numéro']
df_casting_movie = df.copy()[list_casting_movie]

In [177]:
df_casting_movie ["Distribution"] = df_casting_movie["Distribution"].str.split(",\s*")

In [178]:
df_casting_movie  = df_casting_movie.explode("Distribution")

In [179]:
df_casting  = df_casting_movie.explode("Distribution").drop_duplicates("Distribution").reset_index(drop=True).reset_index()
df_casting = df_casting.rename(columns={"index":"id"}).drop(columns="Numéro")

In [180]:
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on = "Distribution").drop(columns="Distribution")

In [181]:
display(df_casting)

,id,Distribution
0,0,Théo Christine
1,1,Sofia Lesaffre
2,2,Finnegan Oldfield
3,3,Jérôme Niel
4,4,Lisa Nyarko
...,...,...
9934,9934,Paulette Goddard
9935,9935,Henry Bergman
9936,9936,Tiny Sandford
9937,9937,Jackie Coogan


In [182]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3635,Vermines,NaN,2023,103,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=vvmsk4CjwIk,NaN,Kaleb est un jeune adulte qui vit de petits bo...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Vermines ...,Matroska (MKV),NaN,Fichier multimédia,2.39 GB,movie/1063879,26744289.0
1,3634,Les derniers samouraïs (vostfr),狼よ落日を斬れ (Titre original),1974,159,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"A la fin du 19ème siècle, un samourai (Sugi To...",[My Book]F:\#DIVX\DIVX MOVIEZ\CHANBARA\The Las...,Matroska (MKV),NaN,Fichier multimédia,1.80 GB,movie/70385,123129.0
2,3633,Un homme nommé Sledge,A Man Called Sledge (Titre original),1970,93,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=Uv430fWuTjU,"Quand SLEDGE frappe la ville, il reste frappé !","Sledge, un célèbre bandit de l'Ouest, apprend ...",[My Book]F:\#DIVX\DIVX MOVIEZ\WESTERN\Un Homme...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/70387,66050.0
3,3632,Sanglantes confessions,True Confessions (Titre original),1981,108,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=iqk_iCcq6PM,Two brothers out to save everyone else. But t...,Tom et Desmond Spellacy sont frères mais ont s...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Sanglantes...,Matroska (MKV),NaN,Fichier multimédia,2.33 GB,movie/32077,83232.0
4,3631,Racket,The Long Good Friday (Titre original),1980,114,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=B90zNzyFk-w,Who lit the fuse that tore Harold's world apart?,"Harold, un gangster anglais prospère, s'apprêt...",[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Racket Du ...,Matroska (MKV),NaN,Fichier multimédia,2.35 GB,movie/14807,81070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3630,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3631,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3632,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3633,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [183]:
display(df_intermediaire_movie_casting)

,id,Numéro
0,0,3635
1,0,1782
2,1,3635
3,1,2024
4,1,1030
...,...,...
17253,9934,2
17254,9935,2
17255,9936,2
17256,9937,1


In [184]:
# Trouver l'index de la ligne avec la 'Distribution' NULL
null_casting_index = df_casting[df_casting['Distribution'].isnull()].index

# Si une ligne avec 'Distribution' NULL est trouvée
if not null_casting_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_casting.loc[null_casting_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_casting['id'].max()
    if max_id > 0:
        df_casting.loc[df_casting.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_casting.head())

# Enregistrer le DataFrame modifié dans la table 'casting' de la base de données
df_casting.to_sql("casting", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on="Distribution").drop(columns="Distribution")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_casting.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_casting.to_sql("movie_casting", index=False, con=engine, if_exists="replace")


     id       Distribution
0  9939     Théo Christine
1     1     Sofia Lesaffre
2     2  Finnegan Oldfield
3     3        Jérôme Niel
4     4        Lisa Nyarko
     id  Numéro
0  9939    3635
1  9939    1782
2     1    3635
3     1    2024
4     1    1030


17258

In [185]:
# df_casting.to_sql("casting",index=False,con = engine,if_exists = "replace" )

In [186]:
# df_intermediaire_movie_casting.to_sql("movie_casting",index=False,con = engine,if_exists = "replace" )

# table movie, country & intermediaire

In [187]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [188]:
list_country_movie = ["Pays", 'Numéro']
df_country_movie = df.copy()[list_country_movie]

In [189]:
df_country_movie ["Pays"] = df_country_movie["Pays"].str.split(",\s*")

In [190]:
df_country_movie  = df_country_movie.explode("Pays")

In [191]:
df_country  = df_country_movie.explode("Pays").drop_duplicates("Pays").reset_index(drop=True).reset_index()
df_country = df_country.rename(columns={"index":"id"}).drop(columns="Numéro")

In [192]:
df_intermediaire_movie_country = df_country.merge(df_country_movie, on = "Pays").drop(columns="Pays")

In [193]:
display(df_country)

,id,Pays
0,0,France
1,1,Japon
2,2,Etats-Unis
3,3,Royaume-Uni
4,4,Hong Kong
5,5,Belgique
6,6,Australie
7,7,Italie
8,8,Inde
9,9,Pologne


In [194]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3635,Vermines,NaN,2023,103,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=vvmsk4CjwIk,NaN,Kaleb est un jeune adulte qui vit de petits bo...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Vermines ...,Matroska (MKV),NaN,Fichier multimédia,2.39 GB,movie/1063879,26744289.0
1,3634,Les derniers samouraïs (vostfr),狼よ落日を斬れ (Titre original),1974,159,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"A la fin du 19ème siècle, un samourai (Sugi To...",[My Book]F:\#DIVX\DIVX MOVIEZ\CHANBARA\The Las...,Matroska (MKV),NaN,Fichier multimédia,1.80 GB,movie/70385,123129.0
2,3633,Un homme nommé Sledge,A Man Called Sledge (Titre original),1970,93,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=Uv430fWuTjU,"Quand SLEDGE frappe la ville, il reste frappé !","Sledge, un célèbre bandit de l'Ouest, apprend ...",[My Book]F:\#DIVX\DIVX MOVIEZ\WESTERN\Un Homme...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/70387,66050.0
3,3632,Sanglantes confessions,True Confessions (Titre original),1981,108,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=iqk_iCcq6PM,Two brothers out to save everyone else. But t...,Tom et Desmond Spellacy sont frères mais ont s...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Sanglantes...,Matroska (MKV),NaN,Fichier multimédia,2.33 GB,movie/32077,83232.0
4,3631,Racket,The Long Good Friday (Titre original),1980,114,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=B90zNzyFk-w,Who lit the fuse that tore Harold's world apart?,"Harold, un gangster anglais prospère, s'apprêt...",[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Racket Du ...,Matroska (MKV),NaN,Fichier multimédia,2.35 GB,movie/14807,81070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3630,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3631,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3632,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3633,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [195]:
display(df_intermediaire_movie_country)

,id,Numéro
0,0,3635
1,0,3618
2,0,3615
3,0,3614
4,0,3605
...,...,...
3630,41,853
3631,42,921
3632,43,769
3633,44,631


In [196]:
# Trouver l'index de la ligne avec le 'Pays' NULL
null_country_index = df_country[df_country['Pays'].isnull()].index

# Si une ligne avec 'Pays' NULL est trouvée
if not null_country_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_country.loc[null_country_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_country['id'].max()
    if max_id > 0:
        df_country.loc[df_country.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_country.head())

# Enregistrer le DataFrame modifié dans la table 'country' de la base de données
df_country.to_sql("country", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_country = df_country.merge(df_country_movie, on="Pays").drop(columns="Pays")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_country.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_country.to_sql("movie_country", index=False, con=engine, if_exists="replace")

   id         Pays
0  45       France
1   1        Japon
2   2   Etats-Unis
3   3  Royaume-Uni
4   4    Hong Kong
   id  Numéro
0  45    3635
1  45    3618
2  45    3615
3  45    3614
4  45    3605


3635

In [197]:
# df_country.to_sql("country",index=False,con = engine,if_exists = "replace" )

In [198]:
df_intermediaire_movie_country.to_sql("movie_country",index=False,con = engine,if_exists = "replace" )

3635

# table movie, genre & intermediaire

In [199]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [200]:
list_genre_movie = ["Genre(s)", 'Numéro']
df_genre_movie = df.copy()[list_genre_movie]

In [201]:
df_genre_movie ["Genre(s)"] = df_genre_movie["Genre(s)"].str.split(",\s*")

In [202]:
df_genre_movie  = df_genre_movie.explode("Genre(s)")

In [203]:
df_genre  = df_genre_movie.explode("Genre(s)").drop_duplicates("Genre(s)").reset_index(drop=True).reset_index()
df_genre = df_genre.rename(columns={"index":"id"}).drop(columns="Numéro")

In [204]:
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on = "Genre(s)").drop(columns="Genre(s)")

In [205]:
display(df_genre)

,id,Genre(s)
0,0,Horreur
1,1,Action
2,2,Aventures
3,3,Chanbara
4,4,Western
5,5,Policier
6,6,Suspense
7,7,Mystère
8,8,Drame
9,9,Histoire


In [206]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3635,Vermines,NaN,2023,103,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=vvmsk4CjwIk,NaN,Kaleb est un jeune adulte qui vit de petits bo...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Vermines ...,Matroska (MKV),NaN,Fichier multimédia,2.39 GB,movie/1063879,26744289.0
1,3634,Les derniers samouraïs (vostfr),狼よ落日を斬れ (Titre original),1974,159,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"A la fin du 19ème siècle, un samourai (Sugi To...",[My Book]F:\#DIVX\DIVX MOVIEZ\CHANBARA\The Las...,Matroska (MKV),NaN,Fichier multimédia,1.80 GB,movie/70385,123129.0
2,3633,Un homme nommé Sledge,A Man Called Sledge (Titre original),1970,93,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=Uv430fWuTjU,"Quand SLEDGE frappe la ville, il reste frappé !","Sledge, un célèbre bandit de l'Ouest, apprend ...",[My Book]F:\#DIVX\DIVX MOVIEZ\WESTERN\Un Homme...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/70387,66050.0
3,3632,Sanglantes confessions,True Confessions (Titre original),1981,108,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=iqk_iCcq6PM,Two brothers out to save everyone else. But t...,Tom et Desmond Spellacy sont frères mais ont s...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Sanglantes...,Matroska (MKV),NaN,Fichier multimédia,2.33 GB,movie/32077,83232.0
4,3631,Racket,The Long Good Friday (Titre original),1980,114,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=B90zNzyFk-w,Who lit the fuse that tore Harold's world apart?,"Harold, un gangster anglais prospère, s'apprêt...",[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Racket Du ...,Matroska (MKV),NaN,Fichier multimédia,2.35 GB,movie/14807,81070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3630,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3631,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3632,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3633,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [207]:
display(df_intermediaire_movie_genre)

,id,Numéro
0,0,3635
1,0,3609
2,0,3607
3,0,3603
4,0,3600
...,...,...
7974,38,920
7975,38,642
7976,38,384
7977,38,352


In [208]:
# Trouver l'index de la ligne avec le 'Genre(s)' NULL
null_genre_index = df_genre[df_genre['Genre(s)'].isnull()].index

# Si une ligne avec 'Genre(s)' NULL est trouvée
if not null_genre_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_genre.loc[null_genre_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_genre['id'].max()
    if max_id > 0:
        df_genre.loc[df_genre.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_genre.head())

# Enregistrer le DataFrame modifié dans la table 'genre' de la base de données
df_genre.to_sql("genre", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on="Genre(s)").drop(columns="Genre(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_genre.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_genre.to_sql("movie_genre", index=False, con=engine, if_exists="replace")


   id   Genre(s)
0  39    Horreur
1   1     Action
2   2  Aventures
3   3   Chanbara
4   4    Western
   id  Numéro
0  39    3635
1  39    3609
2  39    3607
3  39    3603
4  39    3600


7979

In [209]:
df_genre.to_sql("genre",index=False,con = engine,if_exists = "replace" )

39

In [210]:
df_intermediaire_movie_genre.to_sql("movie_genre",index=False,con = engine,if_exists = "replace" )

7979

# table movie, language & intermediaire

In [211]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [212]:
list_language_movie = ["Langues", 'Numéro']
df_language_movie = df.copy()[list_language_movie]

In [213]:
df_language_movie ["Langues"] = df_language_movie["Langues"].str.split(",\s*")

In [214]:
df_language_movie  = df_language_movie.explode("Langues")

In [215]:
df_language  = df_language_movie.explode("Langues").drop_duplicates("Langues").reset_index(drop=True).reset_index()
df_language = df_language.rename(columns={"index":"id"}).drop(columns="Numéro")

In [216]:
df_intermediaire_movie_language = df_language.merge(df_language_movie, on = "Langues").drop(columns="Langues")

In [217]:
display(df_language)

,id,Langues
0,0,Français
1,1,Japonais
2,2,Anglais
3,3,Italien
4,4,Mandarin
5,5,Cantonais
6,6,Thaïlandais
7,7,Espagnol
8,8,Allemand
9,9,NaN


In [218]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3635,Vermines,NaN,2023,103,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=vvmsk4CjwIk,NaN,Kaleb est un jeune adulte qui vit de petits bo...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Vermines ...,Matroska (MKV),NaN,Fichier multimédia,2.39 GB,movie/1063879,26744289.0
1,3634,Les derniers samouraïs (vostfr),狼よ落日を斬れ (Titre original),1974,159,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"A la fin du 19ème siècle, un samourai (Sugi To...",[My Book]F:\#DIVX\DIVX MOVIEZ\CHANBARA\The Las...,Matroska (MKV),NaN,Fichier multimédia,1.80 GB,movie/70385,123129.0
2,3633,Un homme nommé Sledge,A Man Called Sledge (Titre original),1970,93,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=Uv430fWuTjU,"Quand SLEDGE frappe la ville, il reste frappé !","Sledge, un célèbre bandit de l'Ouest, apprend ...",[My Book]F:\#DIVX\DIVX MOVIEZ\WESTERN\Un Homme...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/70387,66050.0
3,3632,Sanglantes confessions,True Confessions (Titre original),1981,108,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=iqk_iCcq6PM,Two brothers out to save everyone else. But t...,Tom et Desmond Spellacy sont frères mais ont s...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Sanglantes...,Matroska (MKV),NaN,Fichier multimédia,2.33 GB,movie/32077,83232.0
4,3631,Racket,The Long Good Friday (Titre original),1980,114,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=B90zNzyFk-w,Who lit the fuse that tore Harold's world apart?,"Harold, un gangster anglais prospère, s'apprêt...",[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Racket Du ...,Matroska (MKV),NaN,Fichier multimédia,2.35 GB,movie/14807,81070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3630,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3631,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3632,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3633,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [219]:
display(df_intermediaire_movie_language)

,id,Numéro
0,0,3635
1,0,3631
2,0,3628
3,0,3618
4,0,3615
...,...,...
5124,42,1060
5125,43,1053
5126,43,930
5127,43,926


In [220]:
# Trouver l'index de la ligne avec les 'Langues' NULL
null_language_index = df_language[df_language['Langues'].isnull()].index

# Si une ligne avec 'Langues' NULL est trouvée
if not null_language_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_language.loc[null_language_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_language['id'].max()
    if max_id > 0:
        df_language.loc[df_language.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_language.head())

# Enregistrer le DataFrame modifié dans la table 'language' de la base de données
df_language.to_sql("language", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_language = df_language.merge(df_language_movie, on="Langues").drop(columns="Langues")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_language.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_language.to_sql("movie_language", index=False, con=engine, if_exists="replace")


   id   Langues
0  45  Français
1   1  Japonais
2   2   Anglais
3   3   Italien
4   4  Mandarin
   id  Numéro
0  45    3635
1  45    3631
2  45    3628
3  45    3618
4  45    3615


5129

In [221]:
# df_language.to_sql("language",index=False,con = engine,if_exists = "replace" )

In [222]:
df_intermediaire_movie_language.to_sql("movie_language",index=False,con = engine,if_exists = "replace" )

5129

# table, movie, tag & intermediaire

In [223]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [224]:
list_tag_movie = ["Etiquettes", 'Numéro']
df_tag_movie = df.copy()[list_tag_movie]

In [225]:
df_tag_movie ["Etiquettes"] = df_tag_movie["Etiquettes"].str.split(",\s*")

In [226]:
df_tag_movie  = df_tag_movie.explode("Etiquettes")

In [227]:
df_tag = df_tag_movie.explode("Etiquettes").drop_duplicates("Etiquettes").reset_index(drop=True).reset_index()
df_tag = df_tag.rename(columns={"index":"id"}).drop(columns="Numéro")

In [228]:
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on = "Etiquettes").drop(columns="Etiquettes")

In [229]:
display(df_tag)

,id,Etiquettes
0,0,NaN
1,1,Chanbara
2,2,Triades
3,3,Mafia
4,4,Voiture
...,...,...
203,203,Momie
204,204,viking
205,205,Réalisme soviétique
206,206,Guy Debord


In [230]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3635,Vermines,NaN,2023,103,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=vvmsk4CjwIk,NaN,Kaleb est un jeune adulte qui vit de petits bo...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Vermines ...,Matroska (MKV),NaN,Fichier multimédia,2.39 GB,movie/1063879,26744289.0
1,3634,Les derniers samouraïs (vostfr),狼よ落日を斬れ (Titre original),1974,159,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"A la fin du 19ème siècle, un samourai (Sugi To...",[My Book]F:\#DIVX\DIVX MOVIEZ\CHANBARA\The Las...,Matroska (MKV),NaN,Fichier multimédia,1.80 GB,movie/70385,123129.0
2,3633,Un homme nommé Sledge,A Man Called Sledge (Titre original),1970,93,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=Uv430fWuTjU,"Quand SLEDGE frappe la ville, il reste frappé !","Sledge, un célèbre bandit de l'Ouest, apprend ...",[My Book]F:\#DIVX\DIVX MOVIEZ\WESTERN\Un Homme...,Matroska (MKV),NaN,Fichier multimédia,1.96 GB,movie/70387,66050.0
3,3632,Sanglantes confessions,True Confessions (Titre original),1981,108,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=iqk_iCcq6PM,Two brothers out to save everyone else. But t...,Tom et Desmond Spellacy sont frères mais ont s...,[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Sanglantes...,Matroska (MKV),NaN,Fichier multimédia,2.33 GB,movie/32077,83232.0
4,3631,Racket,The Long Good Friday (Titre original),1980,114,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=B90zNzyFk-w,Who lit the fuse that tore Harold's world apart?,"Harold, un gangster anglais prospère, s'apprêt...",[My Book]F:\#DIVX\DIVX MOVIEZ\POLAR\Racket Du ...,Matroska (MKV),NaN,Fichier multimédia,2.35 GB,movie/14807,81070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3630,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3631,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3632,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3633,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [231]:
display(df_intermediaire_movie_tag)

,id,Numéro
0,0,3635
1,0,3633
2,0,3632
3,0,3631
4,0,3628
...,...,...
4588,206,74
4589,206,73
4590,207,17
4591,207,16


In [232]:
# Trouver l'index de la ligne avec les 'Etiquettes' NULL
null_tag_index = df_tag[df_tag['Etiquettes'].isnull()].index

# Si une ligne avec 'Etiquettes' NULL est trouvée
if not null_tag_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_tag.loc[null_tag_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_tag['id'].max()
    if max_id > 0:
        df_tag.loc[df_tag.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_tag.head())

# Enregistrer le DataFrame modifié dans la table 'tag' de la base de données
df_tag.to_sql("tag", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on="Etiquettes").drop(columns="Etiquettes")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_tag.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_tag.to_sql("movie_tag", index=False, con=engine, if_exists="replace")


    id Etiquettes
0  208        NaN
1    1   Chanbara
2    2    Triades
3    3      Mafia
4    4    Voiture
    id  Numéro
0  208    3635
1  208    3633
2  208    3632
3  208    3631
4  208    3628


4593

In [233]:
# df_tag.to_sql("tag",index=False,con = engine,if_exists = "replace" )

In [234]:
# df_intermediaire_movie_tag.to_sql("movie_tag",index=False,con = engine,if_exists = "replace" )